### Import Libraries

In [1]:
import sys
import numpy as np

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

### We'd like to solve this linear optimization problem: 
\begin{align*}
	\underset{x}{\min} \quad & \sum_{i=1}^{7} x_i \\
	\text{s.t.} \quad & x_1 + x_4 + x_5 + x_6 + x_7 \geq 100 \\
	& x_1 + x_2 + x_5 + x_6 + x_7 \geq 90 \\
	& x_1 + x_2 + x_3 + x_6 + x_7 \geq 80 \\
	& x_1 + x_2 + x_3 + x_4 + x_7 \geq 80 \\
	& x_1 + x_2 + x_3 + x_4 + x_5 \geq 100 \\
	& x_2 + x_3 + x_4 + x_5 + x_6 \geq 50 \\
	& x_3 + x_4 + x_5 + x_6 + x_7 \geq 30
\end{align*}

In [2]:
model = pyo.ConcreteModel("Nurse Schedule Problem")

# Decision variables and their domains
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)
model.x4 = pyo.Var(domain=pyo.NonNegativeReals)
model.x5 = pyo.Var(domain=pyo.NonNegativeReals)
model.x6 = pyo.Var(domain=pyo.NonNegativeReals)
model.x7 = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
model.profit = pyo.Objective(expr=model.x1 + model.x2 + model.x3 + model.x4 + model.x5 + model.x6 + model.x7, sense=pyo.minimize)

# Constraints
model.Mon = pyo.Constraint(expr=model.x1 + model.x4 + model.x5 + model.x6 + model.x7 >= 100)
model.Tue = pyo.Constraint(expr=model.x1 + model.x2 + model.x5 + model.x6 + model.x7 >= 90)
model.Wed = pyo.Constraint(expr=model.x1 + model.x2 + model.x3 + model.x6 + model.x7 >= 80)
model.Thu = pyo.Constraint(expr=model.x1 + model.x2 + model.x3 + model.x4 + model.x7 >= 80)
model.Fri = pyo.Constraint(expr=model.x1 + model.x2 + model.x3 + model.x4 + model.x5 >= 100)
model.Sat = pyo.Constraint(expr=model.x2 + model.x3 + model.x4 + model.x5 + model.x6 >= 50)
model.Sun = pyo.Constraint(expr=model.x3 + model.x4 + model.x5 + model.x6 + model.x7 >= 30)

# Solve and print solution
SOLVER.solve(model)
print(f"x = ({model.x1.value:.1f}, {model.x2.value:.1f}, {model.x3.value:.1f}, {model.x4.value:.1f}, {model.x5.value:.1f}, {model.x6.value:.1f}, {model.x7.value:.1f})")
print(f"optimal value = {pyo.value(model.profit):.1f}")

x = (60.0, 0.0, 10.0, 10.0, 20.0, 10.0, 0.0)
optimal value = 110.0


### However, a more standard form may help reduce the amount of code. 
$$
\begin{aligned}
\underset{x}{\min} \quad & c^{T}x \\
\text{s.t.} \quad & Ax \geq b \\
& x \geq 0
\end{aligned}
$$
in which
$$
c = \begin{pmatrix} 1 & 1 & 1 & 1 & 1 & 1 & 1 \end{pmatrix}^{T}
$$

$$
b = \begin{pmatrix} 100 & 90 & 80 & 80 & 100 & 50 & 30 \end{pmatrix}^{T}
$$

$$
A = \begin{pmatrix}
    1 & 0 & 0 & 1 & 1 & 1 & 1 \\
    1 & 1 & 0 & 0 & 1 & 1 & 1 \\
    1 & 1 & 1 & 0 & 0 & 1 & 1 \\
    1 & 1 & 1 & 1 & 0 & 0 & 1 \\
    1 & 1 & 1 & 1 & 1 & 0 & 0 \\
    0 & 1 & 1 & 1 & 1 & 1 & 0 \\
    0 & 0 & 1 & 1 & 1 & 1 & 1
\end{pmatrix}
$$


In [3]:
# use Concrete models
model = pyo.ConcreteModel("Nurse Schedule Problem by standard form")

# Define the number of variables and constraints
n_vars = 7
n_constraints = 7

# Decision variables and their domain
model.x = pyo.Var(range(n_vars), domain=pyo.NonNegativeReals)

# Define the vectors and matrices
c = np.array([1, 1, 1, 1, 1, 1, 1])
A = np.array([
    [1, 0, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 1, 1],
    [1, 1, 1, 1, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1]
])
b = np.array([100, 90, 80, 80, 100, 50, 30])

# Objective function
model.profit = pyo.Objective(
    expr=sum(c[i] * model.x[i] for i in range(n_vars)), sense=pyo.minimize
)

# Constraints
model.constraints = pyo.ConstraintList()
for i in range(n_constraints):
    model.constraints.add(expr=sum(A[i, j] * model.x[j] for j in range(n_vars)) >= b[i])
    
# Solve and print solution
SOLVER.solve(model)
optimal_x = [pyo.value(model.x[i]) for i in range(n_vars)]
print(f"x = {tuple(np.round(optimal_x, 1))}")
print(f"optimal value = {pyo.value(model.profit):.1f}")

x = (60.0, 0.0, 10.0, 10.0, 20.0, 10.0, 0.0)
optimal value = 110.0
